In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time
import re
import os

In [2]:
driver = webdriver.Firefox()
driver.get("https://www.imdb.com")
driver.maximize_window()


In [3]:
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")

In [4]:
menu.click()
time.sleep(1)

In [5]:
link_populares = driver.find_element(By.LINK_TEXT, r'250 séries mais populares')

In [6]:
url = link_populares.get_attribute("href")
url

'https://www.imdb.com/pt/chart/toptv/?ref_=nv_tvv_250'

In [7]:
driver.get(url)

In [8]:
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
lista_series = tag_ul.find_elements(By.TAG_NAME, "li")

In [52]:
type(lista_series)

list

In [9]:
series_list = []
i=0
for serie in lista_series:
    i+=1
    print(f"{i}/{len(series_list)}", end="\r")
    titulo = str(serie.find_element(By.CLASS_NAME, "ipc-title__text").text.split(".", 1)[-1]).strip()
    link = serie.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute('href')
    ano = serie.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[1]').text
    imdb = float(serie.find_elements(By.CSS_SELECTOR, ".ipc-rating-star--imdb")[0].text.split("\n")[0].replace(",","."))
    ep = int(serie.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[2]').text.split(" ")[0])
    
    series_list.append({
         "titulo": titulo,
         "anolancamento": ano,
         "episodios": ep,
         "imdb": imdb,
         "link": link
        })

In [10]:
i=0
for serie in series_list:
    i+=1
    print(f"{i}/{len(series_list)}", end="\r")
    try:
        driver.get(serie['link'])
        
        tag = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/div[2]/div/div[3]/a/span/div/div[2]/div').text
        serie['popularidade'] = int(tag.replace(".",""))

        lista_elenco = driver.find_elements(By.CSS_SELECTOR, "[data-testid='title-cast-item']")
        
        elenco = [ator.find_element(By.CSS_SELECTOR, "[data-testid='title-cast-item__actor']").text for ator in lista_elenco]
        serie["atores"] = elenco

        personagens = [personagem.find_elements(By.CSS_SELECTOR, "[data-testid='cast-item-characters-link']")[0].text for personagem in lista_elenco]
        serie["personagens"] = personagens
    except:
        print("Ator ou personagem vazio.")
        continue

Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.
Ator ou personagem vazio.


In [11]:
len(series_list)

250

In [15]:
i=0
for serie in series_list:
    i+=1
    # print(re.sub('[^A-Za-z0-9]+', '', serie.get('titulo')))
    with open(f"jsons/{re.sub('[^A-Za-z0-9]+', '', serie.get('titulo'))}.json", "w", encoding="utf-8") as arquivo:
        print(i)
        json.dump(serie, arquivo, ensure_ascii=False, indent=4)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250


In [23]:
!pwd

/Users/elainelosch/repositories/cpa-webscrapping/t2


In [32]:
existe  = str(os.listdir("jsons"))
for serie in series_list:
    if f"{re.sub('[^A-Za-z0-9]+', '', serie.get('titulo'))}.json" not in existe:
        print(serie.get("titulo"))
    # print(serie.get("titulo"))


In [29]:
len(series_list)

250

In [ ]:
driver.quit()